# T1053 Scheduled Task/Job

-----------------------------------------------------------------------

## Technique Description

Adversaries may abuse task scheduling functionality to facilitate initial or recurring execution of malicious code. Utilities exist within all major operating systems to schedule programs or scripts to be executed at a specified date and time. A task can also be scheduled on a remote system, provided the proper authentication is met (ex: RPC and file and printer sharing in Windows environments). Scheduling a task on a remote system typically may require being a member of an admin or otherwise privileged group on the remote system.(Citation: TechNet Task Scheduler Security)

Adversaries may use task scheduling to execute programs at system startup or on a scheduled basis for persistence. These mechanisms can also be abused to run a process under the context of a specified account (such as one with elevated permissions/privileges). Similar to [System Binary Proxy Execution](https://attack.mitre.org/techniques/T1218), adversaries have also abused task scheduling to potentially mask one-time execution under a trusted system process.(Citation: ProofPoint Serpent)

## Technique Detection

Monitor scheduled task creation from common utilities using command-line invocation. Legitimate scheduled tasks may be created during installation of new software or through system administration functions. Look for changes to tasks that do not correlate with known software, patch cycles, etc. 

Suspicious program execution through scheduled tasks may show up as outlier processes that have not been seen before when compared against historical data. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as network connections made for Command and Control, learning details about the environment through Discovery, and Lateral Movement.

-----------------------------------------------------------------------

### Tactics:

  * Execution

  * Persistence

  * Privilege-Escalation

### Platforms:

  * Windows

  * Linux

  * macOS

  * Containers

### Adversary Required Permissions:

  * Administrator

  * SYSTEM

  * User

### Effective Permissions:

  * SYSTEM

  * Administrator

  * User

### Data Sources:

  * **Scheduled Job:** Scheduled Job Creation

  * **Command:** Command Execution

  * **Container:** Container Creation

  * **Process:** Process Creation

  * **File:** File Creation

  * **File:** File Modification

-----------------------------------------------------------------------

### Adversarial usage:

| Adversary Group |  Adversarial Usage |
|----|----|
-----------------------------------------------------------------------

## Mitre References

  * [Mitre-Attack](https://attack.mitre.org/techniques/T1053)

  * [Proofpoint Serpent](https://www.proofpoint.com/us/blog/threat-insight/serpent-no-swiping-new-backdoor-targets-french-entities-unique-attack-chain), Campbell, B. et al. (2022, March 21). Serpent, No Swiping! New Backdoor Targets French Entities with Unique Attack Chain. Retrieved April 11, 2022.

  * [Technet Task Scheduler Security](https://technet.microsoft.com/en-us/library/cc785125.aspx), Microsoft. (2005, January 21). Task Scheduler and security. Retrieved June 8, 2016.

  * [Capec](https://capec.mitre.org/data/definitions/557.html)

> *Note: Do not edit this cell with information you want to keep. This cell will be wiped when the update script is ran. Store permanent information in one of the relevant cells below*

*Last pulled from Mitre on: 23 June 2022*



-----------------------------------------------------------------------

## Metadata

  * **Last Updated  Date:** 8 July 2021

  * **Author(s):** SSgt Johnathan Smith, SSgt John Beres

  * **Validated:** NO

-----------------------------------------------------------------------

## Overall Hypothesis

- Malicious actors may execute commands leveraging LOLBins to create scheduled tasks.
- Malicious actors may have already created scheduled tasks in the environment.

## Adversary Examples

| Adversary Specific Examples | Host Analytics | Network Analytics |
|-----------------------------|----------------|-------------------|
| Example APT/Threat actor | 1, 2 | 2, 3, 5|

#### APT28	
- used scheduled tasks to establish persistence. -FireEye report "Hunting for APT28"

#### APT29	
- used named and hijacked scheduled tasks to establish persistence.

#### Turla	
- Carbon creates several tasks for later execution to continue persistence on the victim’s machine.
- ComRAT has used a scheduled task to launch its PowerShell loader.
- Gazer can establish persistence by creating a scheduled task

## Detection Blindspots

- Potentially unknown/undocumented scheduled task behaviors. (Risk is very low)

## Analytical References

  * https://docs.microsoft.com/en-us/previous-versions/windows/it-pro/windows-server-2003/cc785125(v=ws.10)?redirectedfrom=MSDN
  * https://attack.mitre.org/groups/G0016/
  * https://github.com/Neo23x0/sigma/blob/master/rules/windows/builtin/win_rare_schtasks_creations.yml
  * https://www.welivesecurity.com/2017/03/30/carbon-paper-peering-turlas-second-stage-backdoor/
  * https://www.welivesecurity.com/wp-content/uploads/2020/05/ESET_Turla_ComRAT.pdf
  * https://technet.microsoft.com/library/dd315590.aspx
  * https://docs.microsoft.com/en-us/windows/security/threat-protection/auditing/audit-other-object-access-events
  * https://technet.microsoft.com/en-us/sysinternals/bb963902
  * https://www.welivesecurity.com/wp-content/uploads/2017/08/eset-gazer.pdf
  * https://securelist.com/introducing-whitebear/81638/
  * https://www.elastic.co/guide/en/siem/guide/current/prebuilt-rules.html

-----------------------------------------------------------------------

## Host Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Several events will then be logged on scheduled task activity, including:
    - Event ID 106 on Windows 7, Server 2008 R2 - Scheduled task registered
    - Event ID 140 on Windows 7, Server 2008 R2 / 4702 on Windows 10, Server 2016 - Scheduled task updated
    - Event ID 141 on Windows 7, Server 2008 R2 / 4699 on Windows 10, Server 2016 - Scheduled task deleted
    - Event ID 4698 on Windows 10, Server 2016 - Scheduled task created
    - Event ID 4700 on Windows 10, Server 2016 - Scheduled task enabled
    - Event ID 4701 on Windows 10, Server 2016 - Scheduled task disabled
- It’s also a good idea to monitor for tasks that spawn at seemingly strange intervals. For example, something like /sc minute /mo 20, which means that the task will run every 20 minutes, should be viewed with suspicion. There are few legitimate reasons that a task would need to run every 20 minutes—or at any other fixed interval.
- The Advanced Audit Policy setting Object Access > Audit Other Object Access Events should be configured to allow this detection
- SIGMA rules can be converted into many different SIEM rules/searches/queries. See https://github.com/Neo23x0/sigma.
- The Elastalert queries above provide a framework for Kibana queries.
- The most common executables for this activity are wmic.exe, at.exe, and schtasks.exe

#### Analytic 1

  * **Information:** Tracking for Scheduled Tasks that load binaries (.exe, for example) or scripts (.ps1, .vbe, and .vbs to name a few) from suspicious paths

  * **Source:** Windows Audits, Sysmon

  * **Tool:** Kibana

  * **Notes:**

  * **Query:** ```Event.action:"Process Create (rule: ProcessCreate)" and process.name:schtasks.exe and process.args:(-change or -create or -run or -s or /S or /change or /create or /run or /xml or /query or /tr or /sc or /mo)```

#### Analytic 2

  * **Information:** at.exe usage

  * **Source:** Windows Audits, Sysmon

  * **Tool:** Kibana

  * **Notes:** 

  * **Query:** ```Event.action:"Process Create (rule: ProcessCreate)" and process.name:at.exe```

#### Analytic 3 
Information: Detect Scheduling Tasks using LOLBins
1. Process start events capture creation of scheduled tasks and windows audit will log scheduled task creation events. This will not occur frequently on a network under normal circumstances 
2. Analysis of scheduled task creation activity should consider the following:
    - How common is this task? Though MCAs may use the same tools to administer the network as admins, do not dismiss high-frequency scheduled tasks outright.
    - Did the creation command construction fit network norms? (flags, syntax, order, etc)
    - What is being scheduled? Basically, did the admins intend to configure the target program to run? For help requesting information from the MP, see the MP Interaction guide.
        - Binaries (.exe, .dll)
        - Scripts (.ps1, .bat, .cmd, .vbe, .vbs, etc)
    - Execution path? The default execution path for tasks created by schtasks.exe or at.exe is 'c:\windows\system32\' and 'c:\windows\syswow64\'
    - Was "/XML" in the command? 
    - Lineage of the scheduling process:
        - Was it launched from a shell?
        - Was it launched by a standard user?
        - Was the executable located in the correct place?
        - Did the scheduling process go on to spawn additional Processes?

#### Analytic 4
Information: Find Already Scheduled Tasks
1. Perform a search to collect scheduled tasks from all or a subset of hosts.
    - This data can be queried using host agents or using a variety of remote scripting capabilities such as PowerShell.
    - Where possible, consider capabilities already present on the system over deploying a more familiar solution.
    - Scheduled tasks are usually found in:
        - Relevant Registry keys
        - Relevant file locations

2. It's recommended to analyze the collected task data in groups based on their originating endpoint class (type and role).
3. Analysis of scheduled tasks should consider the following:
    - How common is this task? Though MCAs may use the same tools to administer the network as admins, do not dismiss high-frequency scheduled tasks outright.
    - What is being scheduled? Basically, did the admins intend to configure the target program to run? For help requesting information from the MP, see the MP Interaction guide.
        - Binaries (.exe, .dll)
        - Scripts (.ps1, .bat, .cmd, .vbe, .vbs, etc)
    - Execution path? The default execution path for tasks created by schtasks.exe or at.exe is 'c:\windows\system32\' and 'c:\windows\syswow64\'
    - Do the collected artifacts have meta-characteristics that stand out?
        - Timestamps (files or registry keys)
        - Permissions (files or registry keys)
        - Owner (files)

-----------------------------------------------------------------------

## Network Analytics

-----------------------------------------------------------------------

### Hunter Notes

- Information Here

#### Analytic 1

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

#### Analytic 2

  * **Information:** 'Comments'

  * **Source:** 'ex Windows Audits, Sysmon'

  * **Tool:** 'Arkime, Kibana, Autopsy'

  * **Notes:** 'Various notes to assist with hunting.'

  * **Query:** ```Input your query here```

